# Xanadu Rewards Application - Deployment Guide

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Step-1-Repository-Setup">Step 1: Repository Setup (Choose Your Deployment Option)</a></li>
<li><a href="#Step-2-Set-Environment-Variables">Step 2: Set Environment Variables</a></li>
<li><a href="#Step-3-Verify-Project-Structure">Step 3: Verify Project Structure</a></li>
<li><a href="#Step-4-Download-Product-Images-and-SQL-Files">Step 4: Download Product Images and SQL Files</a></li>
<li><a href="#Step-5-Prepare-Lambda-Functions-and-Layer">Step 5: Prepare Lambda Functions and Layer</a></li>
<li><a href="#Step-6-Create-S3-Bucket">Step 6: Create S3 Bucket</a></li>
<li><a href="#Step-7-Upload-Lambda-Code-and-Layer-to-S3">Step 7: Upload Lambda Code and Layer to S3</a></li>
<li><a href="#Step-8-Deploy-CloudFormation-Stack">Step 8: Deploy CloudFormation Stack</a></li>
<li><a href="#Step-9-Get-CloudFormation-Outputs">Step 9: Get CloudFormation Outputs</a></li>
<li><a href="#Step-10-Update-Environment-Variables">Step 10: Update Environment Variables</a></li>
<li><a href="#Step-11-Build-the-Frontend">Step 11: Build the Frontend</a></li>
<li><a href="#Step-12-Update-Lambda-Function">Step 12: Update Lambda Function</a></li>
<li><a href="#Step-13-Test-the-API-Endpoints">Step 13: Test the API Endpoints</a></li>
<li><a href="#Step-14-Create-Test-User">Step 14: Create Test User</a></li>
<li><a href="#Step-15-Access-the-Application">Step 15: Access the Application</a></li>
<li><a href="#Step-16-Monitoring-and-Troubleshooting">Step 16: Monitoring and Troubleshooting</a></li>
<li><a href="#Step-17-Cleanup">Step 17: Cleanup</a></li>
</ul>
</div>

This notebook provides step-by-step instructions for deploying the Xanadu Rewards Application on AWS. The application consists of a React frontend and a serverless backend using AWS services.

## Prerequisites

Before starting the deployment, ensure you have the following prerequisites:

### Required Software
1. **AWS CLI** installed and configured with appropriate permissions
2. **Node.js and npm** installed (version 18 or higher recommended)
3. **Git** installed
4. **Python 3** with Jupyter Notebook support

### AWS Account Requirements

**AWS Account** with sufficient permissions to create:
   - VPC and networking resources
   - Aurora PostgreSQL Serverless v2
   - Lambda functions and layers
   - API Gateway
   - Cognito User Pool
   - Amplify hosting
   - S3 buckets
   - CloudFormation stacks

**Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)

### GitHub Integration
1. **GitHub account** with a repository for the application
2. **GitHub OAuth token** with repo access (for Amplify deployment)
   - Go to GitHub Settings > Developer settings > Personal access tokens
   - Generate a token with 'repo' scope

### Cost Considerations
⚠️ **Important**: This deployment creates billable AWS resources. **Estimated monthly cost: $15-75** depending on usage patterns:

**Major cost components:**
- Aurora Serverless v2: $8-40/month (0-4 ACUs, varies by usage)
- NAT Gateway: $32/month (always-on)
- RDS Proxy: $10/month (always-on)
- Lambda, API Gateway, S3, Amplify: $2-8/month (usage-based)
- KMS keys: $4/month (4 keys)

**Cost optimization tips:**
- Aurora scales to 0 ACUs when idle (saves ~$8/month during inactive periods)
- Consider removing NAT Gateway for development environments
- Use [AWS Pricing Calculator](https://calculator.aws) for detailed estimates

**Remember to run the cleanup step when done testing to avoid ongoing charges.**

## Step 1: Repository Setup

Choose one of the following options based on your preferred setup:

### 📋 Repository Setup Options:

#### Option A: Using the full DB Cookbook repository
- Fork/clone the entire db-cookbook repository
- Amplify will build from the subfolder
- Repository: `https://github.com/your-username/db-cookbook`

#### Option B: Create a separate repository for just the rewards app
- Copy the rewards-app-example folder to a new repository
- Amplify will build from the repository root
- Repository: `https://github.com/your-username/rewards-app-example`

### 📝 Instructions:

#### Option A - Using DB Cookbook Repository:
1. Fork the db-cookbook repository on GitHub
2. Clone your fork: `git clone https://github.com/YOUR-USERNAME/db-cookbook.git`
3. Navigate to: `cd db-cookbook/3_Building_Your_First_Serverless_Web_App_with_Aurora/rewards-app-example`
4. Set `GITHUB_REPO = 'db-cookbook'` in Step 2

#### Option B - Separate Repository:
1. Create a new repository on GitHub: `rewards-app-example`
2. Copy the contents of rewards-app-example folder to your new repo
3. Push the code to your repository
4. Set `GITHUB_REPO = 'rewards-app-example'` in Step 2
5. Update CloudFormation template (remove subfolder paths from BuildSpec)

**Run the code cell below to select your option:**

In [ ]:
# Repository Setup - Choose your deployment scenario

# Get user choice
choice = input("Which option are you using? (A/B): ").upper()

if choice == 'A':
    REPO_TYPE = 'cookbook'
    DEFAULT_REPO_NAME = 'db-cookbook'
    print("✓ Using Option A: DB Cookbook repository")
elif choice == 'B':
    REPO_TYPE = 'separate'
    DEFAULT_REPO_NAME = 'rewards-app-example'
    print("✓ Using Option B: Separate repository")
    print("⚠️  Remember to update the CloudFormation template for separate repo!")
else:
    REPO_TYPE = 'cookbook'
    DEFAULT_REPO_NAME = 'db-cookbook'
    print("⚠️  Invalid choice, defaulting to Option A")

## Step 2: Set Environment Variables

This step configures essential environment variables for the deployment including AWS region, stack name, S3 bucket, and GitHub repository details. The script automatically detects your current AWS region and sets up variables for CloudFormation deployment. You'll see the configured values displayed for verification before proceeding.

In [ ]:
# Validate prerequisites and set configuration variables
import os
import uuid
import subprocess
import sys

# Check required tools
def check_tool(tool_name, command):
    try:
        result = subprocess.run(command, capture_output=True, text=True, shell=True)
        if result.returncode == 0:
            print(f"✓ {tool_name} is available")
            return True
        else:
            print(f"✗ {tool_name} is not available or not configured")
            return False
    except Exception as e:
        print(f"✗ {tool_name} check failed: {e}")
        return False

print("Checking prerequisites...")
tools_ok = True
tools_ok &= check_tool("AWS CLI", "aws --version")
tools_ok &= check_tool("Node.js", "node --version")
tools_ok &= check_tool("npm", "npm --version")
tools_ok &= check_tool("zip", "zip --version")

if not tools_ok:
    print("⚠️ Please install missing tools before proceeding.")
else:
    print("✓ All prerequisites are available!")

# Get current AWS region or use default
REGION = !aws configure get region
REGION = REGION[0] if REGION else "us-west-2"

# Generate unique bucket name to avoid conflicts
UNIQUE_ID = str(uuid.uuid4())[:8]

# Set other variables - REPLACE THESE WITH YOUR VALUES
STACK_NAME = "rewards-app-example"
ENVIRONMENT = "dev"
BUCKET_NAME = f"xanadu-rewards-app-{UNIQUE_ID}"  # Unique S3 bucket name
GITHUB_OWNER = "your-github-username"  # ⚠️ REPLACE with your GitHub username

# Set repository name based on chosen option
if 'REPO_TYPE' in locals():
    GITHUB_REPO = DEFAULT_REPO_NAME
else:
    GITHUB_REPO = "db-cookbook"  # Default to cookbook repo

GITHUB_TOKEN = "your-github-oauth-token"  # ⚠️ REPLACE with your GitHub OAuth token

# Validate configuration
if GITHUB_OWNER == "your-github-username" or GITHUB_TOKEN == "your-github-oauth-token":
    print("⚠️ WARNING: Please update GITHUB_OWNER and GITHUB_TOKEN with your actual values!")

if 'REPO_TYPE' in locals() and REPO_TYPE == 'separate':
    print("📝 Note: Using separate repository - ensure CloudFormation template is updated!")

print(f"Configuration:")
print(f"Using region: {REGION}")
print(f"Stack name: {STACK_NAME}")
print(f"S3 bucket: {BUCKET_NAME}")
print(f"Unique ID: {UNIQUE_ID}")
print(f"GitHub Owner: {GITHUB_OWNER}")
print(f"GitHub Repo: {GITHUB_REPO}")

## Step 3: Verify Project Structure

This step verifies that all necessary files are present in the current directory. The project includes all frontend React components, backend Lambda functions, and CloudFormation templates needed for deployment. You'll see a listing of the project structure to confirm all files are available.

In [ ]:
# Verify project structure
import os

# Check if we're in the correct directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# List key directories and files
required_items = ['cfn', 'lambda', 'src', 'package.json', 'scripts']
missing_items = []

for item in required_items:
    if os.path.exists(item):
        print(f"✓ {item} found")
    else:
        print(f"✗ {item} missing")
        missing_items.append(item)

# Check for images and SQL directories
if not os.path.exists('images'):
    print("✓ Images directory not present (will be downloaded in Step 4)")
else:
    print("⚠️ Images directory already exists - will be used for deployment")

if not os.path.exists('lambda/sql'):
    print("✓ SQL directory not present (will be downloaded in Step 4)")
else:
    print("⚠️ SQL directory already exists - will be used for deployment")

if missing_items:
    print(f"Warning: Missing items: {missing_items}")
else:
    print("✓ All required project files found!")

## Step 4: Download Product Images and SQL Files

This step downloads product images and SQL files from [AWS Workshop Studio](https://catalog.workshops.aws/) using the provided URLs. These files are used for the application's product catalog and database initialization, and will be uploaded to S3 in a later step.

- Images URL: `https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/images.zip`
- SQL URL: `https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/sql.zip`

In [ ]:
%%bash
# 1. Download product images if they don't already exist
IMAGES_URL="https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/images.zip"

if [ -d "images" ] && [ "$(ls -A images 2>/dev/null)" ]; then
    echo "✓ Using existing product images in images/ ($(ls images | wc -l) files)"
else
    echo "Downloading product images from provided URL..."
    mkdir -p images
    wget -q $IMAGES_URL -O /tmp/images.zip
    unzip -q /tmp/images.zip -d /tmp
    cp -r /tmp/images/* images/
    rm -rf /tmp/images.zip /tmp/images
    echo "✓ Downloaded $(ls images | wc -l) product images"
fi

# 2. Download SQL files if they don't already exist
SQL_URL="https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/sql.zip"

if [ -d "lambda/sql" ] && [ "$(ls -A lambda/sql 2>/dev/null)" ]; then
    echo "✓ Using existing SQL files in lambda/sql/ ($(ls lambda/sql | wc -l) files)"
    ls -la lambda/sql/
else
    echo "Downloading SQL files from provided URL..."
    mkdir -p lambda/sql
    wget -q $SQL_URL -O /tmp/sql.zip
    unzip -q /tmp/sql.zip -d /tmp
    cp -r /tmp/sql/* lambda/sql/
    rm -rf /tmp/sql.zip /tmp/sql
    echo "✓ Copied $(ls lambda/sql | wc -l) SQL files to lambda/sql/"
    ls -la lambda/sql/
    echo "✓ SQL files prepared and temporary files cleaned up!"
fi

## Step 5: Prepare Lambda Functions and Layer

This step installs Node.js dependencies and packages all Lambda functions and the Lambda layer into ZIP files for AWS deployment. The script builds from source code to ensure security and freshness of dependencies. You'll see npm install output and ZIP file creation confirmations.

⚠️ **Note**: The `image-processor-function` generates temporary pre-signed S3 URLs that expire after a short period for demonstration purpose. For production use, implement permanent URLs using S3 bucket hosting combined with [Amazon CloudFront CDN distribution](https://docs.aws.amazon.com/AmazonCloudFront/latest/DeveloperGuide/distribution-working-with.html) and appropriate IAM/bucket policies for secure, scalable image serving.

In [ ]:
# Build Lambda functions and layer from source
import os

zip_files = [
    'lambda/lambda-layer.zip',
    'lambda/xanadu-app-lambda-functions.zip',
    'lambda/cors-lambda.zip',
    'lambda/db-init-function.zip',
    'lambda/image-processor-function.zip'
]

print("Building Lambda functions from source...")

# Remove existing ZIP files for security
for zip_file in zip_files:
    if os.path.exists(zip_file):
        os.remove(zip_file)
        print(f"✓ Removed existing {zip_file}")

# Install dependencies and create main Lambda function ZIP
print("Installing dependencies for main Lambda function...")
!cd lambda/xanadu-app-lambda-functions && rm -rf node_modules && npm install --production --silent 2>/dev/null || npm install --production

# Create ZIP with explicit inclusion of node_modules
!cd lambda/xanadu-app-lambda-functions && zip -q -r ../xanadu-app-lambda-functions.zip . -x '*.git*'

# Install dependencies and create Lambda Layer ZIP
print("Installing dependencies for Lambda layer...")
!cd lambda/lambda-layer/nodejs && rm -rf node_modules && npm install --production --silent 2>/dev/null || npm install --production
!cd lambda/lambda-layer && zip -q -r ../lambda-layer.zip .

# Create other Lambda function ZIPs
print("Creating other Lambda function packages...")
!cd lambda/cors-function && zip -q -r ../cors-lambda.zip .
!cd lambda/db-init-function && zip -q -r ../db-init-function.zip .
!cd lambda/image-processor-function && zip -q -r ../image-processor-function.zip .

# Verify ZIP files were created and contain node_modules
for zip_file in zip_files:
    if os.path.exists(zip_file):
        size = os.path.getsize(zip_file)
        print(f"✓ {zip_file} created ({size} bytes)")
    else:
        print(f"✗ {zip_file} not found")

# Verify main Lambda ZIP contains dependencies
print("\nVerifying Lambda ZIP contains node_modules:")
!unzip -l lambda/xanadu-app-lambda-functions.zip | grep -q node_modules && echo '✓ node_modules found in ZIP' || echo '⚠️ node_modules not found'

## Step 6: Create S3 Bucket

Create an S3 bucket in your specified region to store Lambda deployment packages and application assets if you don't already have one. The bucket serves as the deployment artifact repository for CloudFormation. You'll see either a successful bucket creation message or an error if the bucket already exists.

In [ ]:
# Create S3 bucket (if it doesn't exist)
import os

# Check if bucket exists
check_result = os.system(f'aws s3api head-bucket --bucket {BUCKET_NAME} --region {REGION} 2>/dev/null')

if check_result == 0:
    print(f"Bucket {BUCKET_NAME} already exists")
else:
    # Create bucket with appropriate configuration
    if REGION == 'us-east-1':
        !aws s3api create-bucket --bucket $BUCKET_NAME --region $REGION
    else:
        !aws s3api create-bucket --bucket $BUCKET_NAME --region $REGION --create-bucket-configuration LocationConstraint=$REGION
    print(f"Created bucket {BUCKET_NAME} in region {REGION}")

## Step 7: Upload Lambda Code and Layer to S3

Upload all packaged Lambda functions, the Lambda layer, SQL files, and product images to your S3 bucket. You'll see upload confirmations and directory listings showing successful file transfers.

In [ ]:
# Upload Lambda ZIP files to S3
print("Uploading Lambda functions...")
!aws s3 cp lambda/lambda-layer.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION --only-show-errors
!aws s3 cp lambda/xanadu-app-lambda-functions.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION --only-show-errors
!aws s3 cp lambda/cors-lambda.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION --only-show-errors
!aws s3 cp lambda/db-init-function.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION --only-show-errors
!aws s3 cp lambda/image-processor-function.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION --only-show-errors

# Upload SQL files to S3
print("Uploading SQL files...")
!aws s3 cp lambda/sql/ s3://$BUCKET_NAME/sql/ --recursive --region $REGION --only-show-errors

# Upload product images to S3
print("Uploading product images...")
!aws s3 cp images/ s3://$BUCKET_NAME/images/ --recursive --region $REGION --only-show-errors

# Verify uploads (summary only)
print("Upload verification:")
!aws s3 ls s3://$BUCKET_NAME/lambda-code/ --region $REGION | wc -l | xargs echo 'Lambda files:'
!aws s3 ls s3://$BUCKET_NAME/sql/ --region $REGION | wc -l | xargs echo 'SQL files:'
!aws s3 ls s3://$BUCKET_NAME/images/ --region $REGION | wc -l | xargs echo 'Image files:'

print("✓ Completed uploading Lambda functions, SQL files, and product images to S3!")

## Step 8: Deploy CloudFormation Stack

Deploy the comprehensive CloudFormation template that creates all AWS resources including Aurora PostgreSQL, Lambda functions, API Gateway, Cognito user pools, and Amplify hosting. The deployment uses the product images we downloaded and uploaded to S3 in the previous steps. You'll see CloudFormation deployment progress and completion status.

⚠️ **Important for Separate Repository Users**: If you chose Option B (separate repository), you need to update the CloudFormation template first. See the code cell below for the required changes.

In [ ]:
# Check if CloudFormation template needs updating for separate repository
if 'REPO_TYPE' in locals() and REPO_TYPE == 'separate':
    print("⚠️  IMPORTANT: You're using a separate repository!")
    print("You need to update the CloudFormation template:")
    print("1. Open cfn/cloudformation-rewards-app.yaml")
    print("2. In the AmplifyApp BuildSpec section, remove these lines:")
    print("   - cd 3_Building_Your_First_Serverless_Web_App_with_Aurora/rewards-app-example")
    print("3. Change baseDirectory from:")
    print("   '3_Building_Your_First_Serverless_Web_App_with_Aurora/rewards-app-example/dist'")
    print("   to: 'dist'")
    print("4. Change cache paths from:")
    print("   '3_Building_Your_First_Serverless_Web_App_with_Aurora/rewards-app-example/node_modules/**/*'")
    print("   to: 'node_modules/**/*'")
    print("Press Enter after making these changes...")
    input()

# Deploy CloudFormation stack
print(f"Deploying CloudFormation stack: {STACK_NAME}")
print(f"This may take 15-20 minutes to complete...")

!aws cloudformation deploy \
  --template-file cfn/cloudformation-rewards-app.yaml \
  --s3-bucket $BUCKET_NAME \
  --stack-name $STACK_NAME \
  --region $REGION \
  --capabilities CAPABILITY_NAMED_IAM \
  --tags CreationSource=aws-database-cookbook-v2025.8 \
  --parameter-overrides \
    EnvironmentName=$ENVIRONMENT \
    S3BucketName=$BUCKET_NAME \
    GitHubOwner=$GITHUB_OWNER \
    GitHubRepo=$GITHUB_REPO \
    GitHubOAuthToken=$GITHUB_TOKEN \
    LambdaLayerS3Key=lambda-code/lambda-layer.zip \
    LambdaCodeS3Key=lambda-code/xanadu-app-lambda-functions.zip \
    LambdaCorsCodeS3Key=lambda-code/cors-lambda.zip \
    LambdaDBInitCodeS3Key=lambda-code/db-init-function.zip \
    LambdaImageUrlCodeS3Key=lambda-code/image-processor-function.zip

print("✓ CloudFormation stack deployment completed!")

## Step 9: Get CloudFormation Outputs

Retrieve important output values from the deployed CloudFormation stack including API endpoints, Amplify URLs, database connection details, and resource identifiers. These outputs are essential for configuring the frontend application and testing the deployment. You'll see a formatted table displaying all stack outputs with their values.

In [ ]:
# Get CloudFormation outputs
!aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs" --output table

## Step 10: Update Environment Variables

The following script automatically extracts CloudFormation stack outputs and updates the frontend application's environment configuration file with the correct API endpoints, authentication settings, and resource identifiers. The automation ensures the React application connects to the correct backend services. You'll see the environment file update confirmation.

In [ ]:
# Make the script executable and run it
!chmod +x scripts/update-env.sh

# Run the script to update .env file
!./scripts/update-env.sh $STACK_NAME $REGION

# Verify .env file was created
import os
if os.path.exists('.env'):
    print("✓ .env file created successfully")
    with open('.env', 'r') as f:
        print("Contents:")
        print(f.read())
else:
    print("✗ .env file not found")

## Step 11: Build the Frontend

The following script triggers an AWS Amplify build job that automatically builds and deploys the React frontend from your GitHub repository. The process includes installing dependencies, building the production bundle, and deploying to the Amplify hosting environment. You'll see build job status updates and completion confirmation.

In [ ]:
# Get Amplify App ID from CloudFormation outputs
AMPLIFY_APP_ID = !aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='AmplifyAppId'].OutputValue" --output text
AMPLIFY_APP_ID = AMPLIFY_APP_ID[0]

# Start a new build job for the branch
print("Starting a new build job for the main branch...")
!aws amplify start-job --app-id $AMPLIFY_APP_ID --branch-name main --job-type RELEASE --region $REGION --output json

# Check Amplify build status in 60s intervals
import json
import time

print(f"Monitoring Amplify build status for app: {AMPLIFY_APP_ID}")

for i in range(10):
    status_output = !aws amplify list-jobs --app-id $AMPLIFY_APP_ID --branch-name main --max-items 1 --region $REGION --query "jobSummaries[0].status" --output text
    
    if status_output and status_output[0] != 'None':
        status = status_output[0]
        print(f"Check {i+1}/10 - Status: {status}")
        if status in ['SUCCEED', 'FAILED', 'CANCELLED']:
            print(f"Build completed with status: {status}")
            break
    else:
        print(f"Check {i+1}/10 - No jobs found")
    
    if i < 9:
        time.sleep(60)

## Step 12: Update Lambda Function (if needed)

This optional step demonstrates how to update Lambda function code after the initial deployment by repackaging the function, uploading to S3, and updating the deployed function. This process is useful for iterative development and bug fixes without full stack redeployment. You'll see the function update confirmation and new version information.

In [ ]:
# Get Lambda function name from CloudFormation outputs
LAMBDA_FUNCTION_NAME = !aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='ApiLambdaFunction'].OutputValue" --output text

# Rebuild Lambda function from source
print("Rebuilding Lambda function from source...")
!cd lambda/xanadu-app-lambda-functions && npm install --production
!cd lambda/xanadu-app-lambda-functions && zip -r ../xanadu-app-lambda-functions.zip .

# Upload to S3
!aws s3 cp lambda/xanadu-app-lambda-functions.zip s3://$BUCKET_NAME/lambda-code/ --region $REGION

# Update the Lambda function
!aws lambda update-function-code \
  --function-name $LAMBDA_FUNCTION_NAME \
  --s3-bucket $BUCKET_NAME \
  --s3-key lambda-code/xanadu-app-lambda-functions.zip \
  --region $REGION \
  --publish

print(f"Lambda function {LAMBDA_FUNCTION_NAME} updated successfully")

## Step 13: Test the API Endpoints

This step validates the deployed API by testing key endpoints including products and categories to ensure the backend services are functioning correctly. The tests verify database connectivity, Lambda function execution, and API Gateway routing. You'll see sample API responses demonstrating successful deployment and data retrieval.

In [ ]:
# Get API endpoint and API key from CloudFormation outputs
API_ENDPOINT = !aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='ApiEndpoint'].OutputValue" --output text
API_KEY = !aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='ApiKey'].OutputValue" --output text

API_ENDPOINT = API_ENDPOINT[0] if API_ENDPOINT else 'Not found'
API_KEY_ID = API_KEY[0] if API_KEY else 'Not found'

print(f"API Endpoint: {API_ENDPOINT}")
print(f"API Key ID: {API_KEY_ID}")

# Get the actual API key value
if API_KEY_ID != 'Not found':
    API_KEY_VALUE = !aws apigateway get-api-key --api-key $API_KEY_ID --include-value --region $REGION --query 'value' --output text
    API_KEY_VALUE = API_KEY_VALUE[0] if API_KEY_VALUE else 'Not found'
    print(f"API Key Value: {API_KEY_VALUE[:10]}...")
else:
    API_KEY_VALUE = 'Not found'

# Test products endpoint with API key
print("\nTesting API endpoints with API key...")
import json

if API_ENDPOINT != 'Not found' and API_KEY_VALUE != 'Not found':
    # Test products endpoint
    products_output = !curl -s -H "Content-Type: application/json" -H "X-API-Key: $API_KEY_VALUE" $API_ENDPOINT/prod/products
    
    try:
        products_response = products_output[0] if products_output else '{}'
        products = json.loads(products_response)
        
        if "products" in products and len(products["products"]) > 0:
            print(f"✓ Products API working - found {len(products['products'])} products")
            print("Sample product:")
            print(json.dumps(products["products"][0], indent=2))
        else:
            print(f"⚠️ Products API response: {products_response[:200]}")
    except Exception as e:
        print(f"⚠️ Products API test failed: {str(e)}")
        print(f"Response: {products_response[:200] if 'products_response' in locals() else 'No response'}")
    
    # Test categories endpoint
    print("\nTesting categories endpoint...")
    categories_output = !curl -s -H "Content-Type: application/json" -H "X-API-Key: $API_KEY_VALUE" $API_ENDPOINT/prod/category
    
    try:
        categories_response = categories_output[0] if categories_output else '{}'
        categories = json.loads(categories_response)
        print("✓ Categories API working:")
        print(json.dumps(categories, indent=2))
    except Exception as e:
        print(f"⚠️ Categories API test failed: {str(e)}")
        print(f"Response: {categories_response[:200] if 'categories_response' in locals() else 'No response'}")
else:
    print("⚠️ API endpoint or API key not found in CloudFormation outputs")

## Step 14: Create Test User

This step creates a test user in Cognito User Pool using a random username from the Aurora database. The user will be created without email verification for testing purposes.

In [ ]:
import boto3

# Get CloudFormation outputs
cf = boto3.client('cloudformation', region_name=REGION)
response = cf.describe_stacks(StackName=STACK_NAME)
outputs = {o['OutputKey']: o['OutputValue'] for o in response['Stacks'][0]['Outputs']}

# Get cluster ARN from Step 13 or construct it
cluster_endpoint = outputs.get('ClusterEndpoint', '')
if cluster_endpoint:
    cluster_id = cluster_endpoint.split('.')[0]
    account_id = boto3.client('sts').get_caller_identity()['Account']
    cluster_arn = f"arn:aws:rds:{REGION}:{account_id}:cluster:{cluster_id}"
else:
    cluster_arn = f"arn:aws:rds:{REGION}:{boto3.client('sts').get_caller_identity()['Account']}:cluster:{STACK_NAME}-{ENVIRONMENT}-cluster"
secret_arn = outputs['DBMasterUserSecretArn']
user_pool_id = outputs['UserPoolId']

# Get random username from database
rds_data = boto3.client('rds-data', region_name=REGION)
result = rds_data.execute_statement(
    resourceArn=cluster_arn,
    secretArn=secret_arn,
    database='postgres',
    sql="SELECT username FROM xpoints.customers ORDER BY random() LIMIT 1;"
)

username = result['records'][0][0]['stringValue'] if result['records'] else "diego_ramirez_130@example.com"

# Create Cognito user
cognito = boto3.client('cognito-idp', region_name=REGION)
try:
    cognito.admin_create_user(
        UserPoolId=user_pool_id,
        Username=username,
        TemporaryPassword='TempPass123!',
        MessageAction='SUPPRESS',
        UserAttributes=[
            {'Name': 'email', 'Value': username},
            {'Name': 'email_verified', 'Value': 'true'}
        ]
    )
    
    cognito.admin_set_user_password(
        UserPoolId=user_pool_id,
        Username=username,
        Password='TestPass123!',
        Permanent=True
    )
    
    print(f"✅ Created test user: {username}")
    print(f"   Password: TestPass123!")
    
except Exception as e:
    print(f"❌ Error creating user: {e}")

## Step 15: Access the Application

Retrieve the public URL of your deployed Amplify application where users can access the complete Xanadu Rewards web application. The URL provides access to the fully functional React frontend connected to your serverless backend infrastructure. You'll see the clickable application URL for immediate testing.

In [ ]:
# Get Amplify URL from CloudFormation outputs
print(f"✅ Created test user: {username}")
print(f"   Password: TestPass123!")
print("Use the user and password above for testing purpose")

app_url=!aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='AmplifyURL'].OutputValue" --output text

print(f"✅ Click Application URL to start exploring: {app_url[0]}")

## Step 16: Monitoring and Troubleshooting

This step demonstrates how to access CloudWatch logs for your Lambda functions to monitor application performance and troubleshoot issues. The commands show how to find log groups, identify recent log streams, and retrieve detailed execution logs. You'll see log group information and recent Lambda execution details for debugging purposes.

In [ ]:
# Get Lambda function name from CloudFormation outputs
LAMBDA_FUNCTION = !aws cloudformation describe-stacks --stack-name $STACK_NAME --region $REGION --query "Stacks[0].Outputs[?OutputKey=='ApiLambdaFunction'].OutputValue" --output text

# View Lambda logs
!aws logs describe-log-groups --log-group-name-prefix "/aws/lambda/$LAMBDA_FUNCTION" --region $REGION

# Get the latest log stream
LOG_STREAM = !aws logs describe-log-streams --log-group-name "/aws/lambda/$LAMBDA_FUNCTION" --region $REGION --order-by LastEventTime --descending --limit 1 --query "logStreams[0].logStreamName" --output text

# View logs
!aws logs get-log-events --log-group-name "/aws/lambda/$LAMBDA_FUNCTION" --log-stream-name "$LOG_STREAM" --region $REGION

## Step 17: Cleanup

This cleanup step safely removes all AWS resources created during the deployment by deleting the CloudFormation stack, which automatically handles resource dependencies and cleanup order. The process includes waiting for complete stack deletion to ensure all resources are properly removed. You'll see deletion progress and completion confirmation.

In [ ]:
# Delete main stack
!aws cloudformation delete-stack --stack-name $STACK_NAME --region $REGION

# Wait for stack deletion to complete
!aws cloudformation wait stack-delete-complete --stack-name $STACK_NAME --region $REGION

## Conclusion

You have successfully deployed the Xanadu Rewards Application on AWS with enhanced security features. The application includes:

1. A React frontend hosted on AWS Amplify
2. A serverless backend using AWS Lambda and API Gateway
3. Authentication using Amazon Cognito
4. Database using Amazon Aurora PostgreSQL Serverless v2
5. Connection pooling using Amazon RDS Proxy

### Security Enhancements Implemented:
- ✓ Lambda functions built from source code (no pre-built packages)
- ✓ Product images downloaded from AWS Workshop Studio during deployment
- ✓ SQL files downloaded from official workshop source
- ✓ No sensitive files stored in repository
- ✓ Fresh dependency installation for each deployment

You can now access the application using the Amplify URL and sign in with the test user you created.

## Additional Resources 📚

### Deployment & Infrastructure
- [AWS Amplify Documentation](https://docs.aws.amazon.com/amplify/)
- [CloudFormation User Guide](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/)
- [AWS Lambda Developer Guide](https://docs.aws.amazon.com/lambda/latest/dg/)

### Database & Backend
- [Aurora Serverless v2](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/aurora-serverless-v2.html)
- [RDS Proxy](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/rds-proxy.html)
- [API Gateway](https://docs.aws.amazon.com/apigateway/latest/developerguide/)

### Authentication & Security
- [Amazon Cognito](https://docs.aws.amazon.com/cognito/latest/developerguide/)
- [IAM Best Practices](https://docs.aws.amazon.com/IAM/latest/UserGuide/best-practices.html)
- [AWS Security Best Practices](https://aws.amazon.com/architecture/security-identity-compliance/)